In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd
import numpy as np
import os

sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")

# auto-width
from auto_width8 import auto_columns_width


from universal_connection import UniversalConnection, DBType
from report_utils import query_by_name
from log import log

In [2]:
from region import get_region_by_two_bases, clean_region

In [3]:
conn = UniversalConnection('../../../../.credentials/telemed/prom.cfg', DBType.Postgres)
df: pd.DataFrame = query_by_name(conn, os.getcwd(), 'sql_all')
conn.close()

2020-07-21 14:17:11 connect to postgres database using config file "../../../../.credentials/telemed/prom.cfg"
2020-07-21 14:17:11 creating ssh tunnel to 172.16.100.19 as root...
2020-07-21 14:17:11 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 35667
2020-07-21 14:17:12 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-07-21 14:17:12 @telemed query:
                    with temp as (
				    select	product.id,
				    		product.partner_id,
				    		product.full_name,
				    		pc.id product_condition_id,
				    		promo.id promo_id,
				    		pp.patient_id patient_id--,
				    		--a.id app_id,
				    		--a.price ,
				    		--product.full_name
				    		--concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
				            --p_patient.birthday patient_birthday,
				            --p_patient.phone,
	

In [4]:
df.iloc[0]

patient_id           NaN
patient_fio             
patient_birthday    None
phone               None
email               None
sex                 None
app_id               NaN
full_name           None
cons_date           None
specialty           None
Name: 0, dtype: object

In [5]:
df = df[1:].reset_index(drop=True)
df['app_id'].fillna(0, inplace=True)
df['phone'].fillna('', inplace=True)
df['email'].fillna('', inplace=True)
df['sex'].fillna('', inplace=True)
df['full_name'].fillna('', inplace=True)


df.patient_id = df.patient_id.astype(int)
df.app_id = df.app_id.astype(int)

df['dirty_region'] = df['phone'].apply(get_region_by_two_bases)
df['region'] = df['dirty_region'].apply(clean_region)

In [6]:
df

,patient_id,patient_fio,patient_birthday,phone,email,sex,app_id,full_name,cons_date,specialty,dirty_region,region
0,3501140,Буланкина Екатерина Михайловна,1991-05-27,79638058150,,FEMALE,151805,Телемедицина_ВЭБ,2020-04-20,Акушер-гинеколог,Амурская обл.,Амурская область
1,4654090,Галицкая Ольга Борисовна,1959-05-04,79641183900,galitskayaolga.59@gmail.com,FEMALE,275480,Телемедицина_ВЭБ,2020-06-20,Терапевт,Иркутская обл.,Иркутская область
2,4654090,Галицкая Ольга Борисовна,1959-05-04,79641183900,galitskayaolga.59@gmail.com,FEMALE,275489,Телемедицина_ВЭБ,2020-06-22,Дерматолог,Иркутская обл.,Иркутская область
3,4654090,Галицкая Ольга Борисовна,1959-05-04,79641183900,galitskayaolga.59@gmail.com,FEMALE,278161,Телемедицина_ВЭБ,2020-06-23,Невролог,Иркутская обл.,Иркутская область
4,4654090,Галицкая Ольга Борисовна,1959-05-04,79641183900,galitskayaolga.59@gmail.com,FEMALE,282083,Телемедицина_ВЭБ,2020-06-24,Кардиолог,Иркутская обл.,Иркутская область
...,...,...,...,...,...,...,...,...,...,...,...,...
144679,3747036,К К П,1998-01-14,79678741417,kopalkinaksenia@gmail.com,FEMALE,226560,Телемедицина_ВЭБ,2020-05-25,Терапевт,Пермский край,Пермский край
144680,3747036,К К П,1998-01-14,79678741417,kopalkinaksenia@gmail.com,FEMALE,231969,Телемедицина_ВЭБ,2020-05-26,Акушер-гинеколог,Пермский край,Пермский край
144681,3747036,К К П,1998-01-14,79678741417,kopalkinaksenia@gmail.com,FEMALE,283204,Телемедицина_ВЭБ,2020-06-24,Терапевт,Пермский край,Пермский край
144682,3152772,Потылицина Елена Николаевна,1983-01-06,79503026615,,FEMALE,242333,Телемедицина_ВЭБ,2020-06-01,Невролог,Республика Хакасия,Республика Хакасия


In [7]:
len(df.patient_id.unique())

77968

In [8]:
test = df[df['app_id'] > 0]#
test.groupby(['app_id'])[['patient_id']].count()#.reset_index().rename(columns={'patient_id':'quantity'})

,patient_id
app_id,
116252,1
118293,1
118296,1
120237,1
123496,1
...,...
303145,1
303147,1
303149,1


In [9]:
test = df[df['app_id'] > 0].groupby(['app_id'])[['patient_id']].count().reset_index().rename(columns={'patient_id':'quantity'})

In [10]:
test[test['quantity'] > 1]

,app_id,quantity


In [11]:
df.specialty = df.specialty.str.replace('\<', '').str.replace('\>', '').str.replace('.', '')
df.specialty = df.specialty.str.replace('-ТМ', '')
df.specialty = df.specialty.fillna('Unknown')

In [12]:
#df

In [13]:
spec_list = list(df.specialty.unique())
spec_list.sort()
spec_list.insert(0, 'Всего консультаций')
spec_list

['Всего консультаций',
 'Unknown',
 'Акушер-гинеколог',
 'Аллерголог-иммунолог',
 'Врач УЗИ',
 'Врач общей практики (Семейный врач)',
 'Гастроэнтеролог',
 'Гематолог',
 'Дерматолог',
 'Детский гастроэнтеролог',
 'Детский кардиолог',
 'Диетолог',
 'Инструктор по ЗОЖ и фитнесу',
 'Кардиолог',
 'Консультант по питанию',
 'Косметолог',
 'Невролог',
 'Онколог',
 'Онкология',
 'Оториноларинголог',
 'Офтальмолог',
 'Педиатр',
 'Психолог',
 'Психотерапевт',
 'Пульмонолог',
 'Ревматолог',
 'Ревматология',
 'Специалист по диагностике',
 'Стоматолог терапевт',
 'Терапевт',
 'Травматолог-ортопед',
 'УЗИ',
 'Уролог',
 'Хирург',
 'Эндокринолог']

In [14]:
len(df.patient_id.unique())

77968

In [15]:
d_spec = dict()

for pat_id in list(df.patient_id.unique()):
    
    temp = df[df['patient_id'] == pat_id].reset_index(drop=True)
    #print(d_spec)
    
    d_spec[temp['patient_id'][0]] = d_spec.get(temp['patient_id'][0], [0 for specialization in spec_list])
    
    if temp['app_id'][0] != 0:
        d_spec[temp['patient_id'][0]][0] += 1
        d_spec[temp['patient_id'][0]][spec_list.index(temp['specialty'][0])] += 1 

    if temp.shape[0] > 1:
        
        for i in range(1, temp.shape[0]):
            
            if temp['app_id'][i] != 0:
                
                d_spec[temp['patient_id'][0]][0] += 1
                d_spec[temp['patient_id'][i]][spec_list.index(temp['specialty'][i])] += 1  

            
spec_report = pd.DataFrame.from_dict(d_spec, orient='index').sort_index()
spec_report.columns = spec_list
spec_report = spec_report.reset_index().rename(columns={'index':'patient_id'})
spec_report = spec_report.reset_index(drop=True)
spec_report.drop('Unknown', axis=1, inplace=True)
            
#spec_report.reset_index().head()

In [16]:
spec_report

,patient_id,Всего консультаций,Акушер-гинеколог,Аллерголог-иммунолог,Врач УЗИ,Врач общей практики (Семейный врач),Гастроэнтеролог,Гематолог,Дерматолог,Детский гастроэнтеролог,...,Ревматолог,Ревматология,Специалист по диагностике,Стоматолог терапевт,Терапевт,Травматолог-ортопед,УЗИ,Уролог,Хирург,Эндокринолог
0,611,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,973,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,977,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,979,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,981,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77963,4955655,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77964,4955695,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77965,4955705,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77966,4955740,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df.groupby(['patient_id', 'patient_fio', 'patient_birthday', 'phone', 'email', 'sex', 'region'])[['app_id']].count().reset_index().drop('app_id', axis=1)

,patient_id,patient_fio,patient_birthday,phone,email,sex,region
0,611,Лебедева Ольга Сергеевна,2017-08-21,79067166882,oslebedeva@mail.ru,FEMALE,Москва и Московская область
1,973,Арутюнян Айрапет Люксимбургович,2000-01-01,79300889210,tucoom@mail.ru,MALE,Белгородская область
2,977,Рыбалко Анна Александровна,1989-12-30,79207884295,arubalko@yandex.ru,FEMALE,Тульская область
3,979,Жидкова Людмила Андреевна,1989-04-04,79282144524,lyudmila.jidkova2015@yandex.ru,FEMALE,Ростовская область
4,981,Магомедова Марьян Магомедовна,1982-02-19,79034777877,marian0909098476@mail.ru,FEMALE,Республика Дагестан
...,...,...,...,...,...,...,...
77963,4955655,Кудрявцева Виктория,1994-03-12,79513744965,yeezyeezy@gmail.com,,Новосибирская область
77964,4955695,Юзвенко Максим Юрьевич,1989-07-21,79507427475,,MALE,Челябинская область
77965,4955705,Калинина Алина Калина,1999-09-09,9882058425,ar.mma@mail.ru,FEMALE,Неизвестный регион
77966,4955740,Макарова Ольга Алексеева,1972-12-20,79108026358,,FEMALE,Костромская область


In [18]:
df_pat = df.groupby(['patient_id', 'patient_fio', 'patient_birthday', 'phone', 'email', 'sex', 'region'])[['app_id']].count().reset_index().drop('app_id', axis=1)

In [19]:
df_total = df_pat.merge(spec_report)

In [20]:
global writer
writer = None

writer = auto_columns_width(os.getcwd() + '/data_all' + '.xlsx',
                            df.sort_values(by='patient_id'), 
                            'Данные',
                            writer,
                            1.2)

writer = auto_columns_width(os.getcwd() + '/data_all' + '.xlsx',
                            df_total, 
                            'Данные по врачам',
                            writer,
                            1.2)

writer.save()

In [26]:
df[df['specialty'] == 'Педиатр'].groupby(['patient_id', 'patient_fio', 
                                          'patient_birthday', 'phone',
                                         'email', 'sex', 'region'
                                         ])[['app_id']].count().reset_index()

,patient_id,patient_fio,patient_birthday,phone,email,sex,region,app_id
0,2383,Ковалева Лариса Валерьевна,2017-12-23,79161322271,6161757@gmail.com,FEMALE,Москва и Московская область,1
1,2820,Мельникова Ееатерина Викторовна,1993-11-30,79189179105,93katy@bk.ru,FEMALE,Краснодарский край,1
2,6004,Кузнецова Виктория Викторовна,1989-10-02,79244508614,vika21089@mail.ru,FEMALE,Республика Бурятия,2
3,7220,Сагундукова Зульфия Альбертовна,1992-04-24,79600348591,zulfiya.galieva@gmai.com,FEMALE,Республика Татарстан,1
4,7941,Шокель Антон Владимирович,1980-03-21,79190823481,antonshokel@gmail.com,MALE,Тульская область,1
...,...,...,...,...,...,...,...,...
8195,4929728,Матвеева Ксения Михайловна,1983-04-27,79629000700,shanno4ka@mail.ru,FEMALE,Москва и Московская область,1
8196,4929961,Калмык Ольга Вадимовна,1989-08-22,79897673477,22081989.89kalmik@gmail.com,FEMALE,Краснодарский край,1
8197,4931473,Никульшина Евгения Александровна,1993-04-10,79515244399,,FEMALE,Ростовская область,1
8198,4934399,Кузнецова Владлена Витальевна,1995-10-16,79925180004,ciambella585@icloud.com,FEMALE,Челябинская область,1


In [29]:
df_pedi = df[df['specialty'] == 'Педиатр'].groupby(['patient_id', 'patient_fio', 
                                          'patient_birthday', 'phone',
                                         'email', 'sex', 'region'
                                         ])[['app_id']].count().reset_index().rename(columns={'app_id':'cons_amount'})

In [30]:
df_pedi[df_pedi['cons_amount'] == 1]

,patient_id,patient_fio,patient_birthday,phone,email,sex,region,cons_amount
0,2383,Ковалева Лариса Валерьевна,2017-12-23,79161322271,6161757@gmail.com,FEMALE,Москва и Московская область,1
1,2820,Мельникова Ееатерина Викторовна,1993-11-30,79189179105,93katy@bk.ru,FEMALE,Краснодарский край,1
3,7220,Сагундукова Зульфия Альбертовна,1992-04-24,79600348591,zulfiya.galieva@gmai.com,FEMALE,Республика Татарстан,1
4,7941,Шокель Антон Владимирович,1980-03-21,79190823481,antonshokel@gmail.com,MALE,Тульская область,1
5,7965,Нургалеева Эльвира Ануфовна,1984-05-18,79273091006,elcha_@list.ru,FEMALE,Республика Башкортостан,1
...,...,...,...,...,...,...,...,...
8195,4929728,Матвеева Ксения Михайловна,1983-04-27,79629000700,shanno4ka@mail.ru,FEMALE,Москва и Московская область,1
8196,4929961,Калмык Ольга Вадимовна,1989-08-22,79897673477,22081989.89kalmik@gmail.com,FEMALE,Краснодарский край,1
8197,4931473,Никульшина Евгения Александровна,1993-04-10,79515244399,,FEMALE,Ростовская область,1
8198,4934399,Кузнецова Владлена Витальевна,1995-10-16,79925180004,ciambella585@icloud.com,FEMALE,Челябинская область,1


In [34]:
global writer
writer = None

writer = auto_columns_width(os.getcwd() + '/data_pediatrist_full' + '.xlsx',
                            #df_pedi[df_pedi['cons_amount'] == 1].reset_index(drop=True),
                            df[(df['patient_id'].isin(df_pedi[df_pedi['cons_amount'] == 1].patient_id)) & (df['specialty'] == 'Педиатр')],
                            'Педиатр_1_консультация',
                            writer,
                            1.2)

writer.save()

In [33]:
df[(df['patient_id'].isin(df_pedi[df_pedi['cons_amount'] == 1].patient_id)) & (df['specialty'] == 'Педиатр')]

,patient_id,patient_fio,patient_birthday,phone,email,sex,app_id,full_name,cons_date,specialty,dirty_region,region
32,3534868,Опарина Анна Вячеславовна,1989-03-18,79090900498,,FEMALE,173034,Телемедицина_ВЭБ,2020-04-30,Педиатр,Челябинская обл.,Челябинская область
33,3553052,Гаврилова Анна Анатольевна,1989-08-01,79281267335,povareshka@bk.ru,FEMALE,183469,Телемедицина_ВЭБ,2020-05-05,Педиатр,Ростовская обл.,Ростовская область
42,3524378,Куприянова Ольга Алексеевна,1995-09-09,79059680535,kolevatova09.09@mail.ru,FEMALE,218786,Телемедицина_ВЭБ,2020-05-21,Педиатр,Новосибирская область,Неизвестный регион
57,3505739,Поскребышева Мария Николаевна,1984-02-23,79870510826,,FEMALE,187349,Телемедицина_ВЭБ,2020-05-07,Педиатр,Республика Башкортостан,Республика Башкортостан
78,3032425,Самарина Елена Васильевна,1980-03-07,79508458902,elena.7780@bk.ru,FEMALE,238807,Телемедицина_ВЭБ,2020-05-29,Педиатр,Ростовская обл.,Ростовская область
...,...,...,...,...,...,...,...,...,...,...,...,...
144575,80566,Максимова Ульяна Владимировна,1991-03-09,79299392129,maksimovauv@mail.ru,FEMALE,203185,Телемедицина_ВЭБ,2020-05-16,Педиатр,г. Москва и Московская область,Москва и Московская область
144579,2503771,Мадильян Диана Давидовна,1995-10-04,79183266569,di.abh@mail.ru,FEMALE,200071,Телемедицина_ВЭБ,2020-05-14,Педиатр,Краснодарский край,Краснодарский край
144639,3769855,Акшаев Иван Андреевич,2019-06-28,79603359289,,MALE,245571,Телемедицина_ВЭБ,2020-06-02,Педиатр,Республика Мордовия,Республика Мордовия
144665,3564185,Горина Наталья Александровна,1994-04-04,79068358729,19osj34@mail.ru,FEMALE,196313,Телемедицина_ВЭБ,2020-05-12,Педиатр,Оренбургская обл.,Оренбургская область
